In [1]:
#честно говоря, я так и не пояла, с чем это все работает, а с чем нет, поэтому оставила все
#чекни, без чего работает, а без чего нет
!pip install OSMPythonTools
!pip install -U setuptools
!apt install libspatialindex-c4v5
!pip install rtree
!pip install pipwin
!pip install --upgrade pip
!pip install git+git://github.com/geopandas/geopandas.git
!apt install libspatialindex-c4v5
!pip install pysal

!pip install git+git://github.com/geopandas/geopandas.git
!pip3 install overpy

     |████████████████████████████████| 184kB 5.7MB/s 
  Created wheel for OSMPythonTools: filename=OSMPythonTools-0.2.9-cp36-none-any.whl size=12771 sha256=fa5937ce687eb558938a2ca04e63bc1ddedf5c79865dadbbcebd6ec5132e785b
  Stored in directory: /root/.cache/pip/wheels/4e/65/34/afe0b46e3dbb1b3da010967781d4999084bfe5c63d7b2eeefd
  Created wheel for pytest-sugar: filename=pytest_sugar-0.9.4-py2.py3-none-any.whl size=8971 sha256=5504014d33043745628b03baeb16d51a9980f4d621fd97a2abb76dbb61826fb8
  Stored in directory: /root/.cache/pip/wheels/72/ac/7f/55050ee786fa4a2bb2d05dea0696eab826ff1d5b8a7dbd5883
Successfully built OSMPythonTools pytest-sugar
     |████████████████████████████████| 2.0MB 5.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 50.3.2
    Uninstalling setuptools-50.3.2:
      Successfully uninstalled setuptools-50.3.2


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 270 kB of archives.
After this operation, 1,107 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Fetched 270 kB in 1s (411 kB/s)
Selecting previously unselected package libspatialindex4v5:amd64.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../libspatialindex4v5_1.8.5-5_amd64.deb ...
Unpacking libspatialindex4v5:amd64 (1.8.5-5) ...
Selecting previously unselected package libspatialindex-c4v5:amd64.
Preparing to u

  Cloning git://github.com/geopandas/geopandas.git to /tmp/pip-req-build-dgnsplgx
     |████████████████████████████████| 41 kB 387 kB/s 
  Created wheel for overpy: filename=overpy-0.4-py3-none-any.whl size=45771 sha256=15da890d8836b3adb4e0d15b581fc5759b04bf611157f8db0423f360208f0f93
  Stored in directory: /root/.cache/pip/wheels/be/26/99/bcaf5f060042530729608b5714e962d6b245f83cd986ed127b
Successfully built overpy


### Задача:

Прогноз продаж одной из популярных моделей [фичерфонов](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D1%87%D0%B5%D1%80%D1%84%D0%BE%D0%BD) (на картинке ниже пример похожего устройства) в салонах МегаФона
![](https://39.img.avito.st/640x480/8468720439.jpg)

### Исходные данные:

Датасет содержит следующие поля:

1. `point_id` - Индентификатор салона
2. `lon` - Долгота точки
3. `lat` - Широта точки
4. `target` - Значение таргета, усредненное за несколько месяцев и отнормированное

### Требования к решению и советы:

Ниже приведен список из нескольких важных пунктов, необходимых для решения задания. Выполнение каждого из пунктов влияет на итоговую оценку. Вы можете выполнить каждый из пунктов разными способами, самым лучшим будет считаться вариант, когда всё получение и обработка данных будут реализованы на Питоне (пример: вы можете скачать данные из OSM через интерфейс на сайте overpass-turbo или с помощью библиотек `overpass`/`requests`. Оба варианта будут зачтены, но больше баллов можно заработать во втором случае)



1. Салоны расположены в нескольких разных городах, вам необходимо **определить город для каждого салона** (это понадобится во многих частях задания). К этому есть разные подходы. Вы можете провести [обратное геокодирование](https://en.wikipedia.org/wiki/Reverse_geocoding) с помощью геокодера [Nominatim](https://nominatim.org/), доступного через библиотеку `geopy` примерно вот так:
```python
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse("52.509669, 13.376294")
print(location.address)
```
В таком случае, вам придется обрабатывать полученную строку адреса, чтобы извлечь название города. Также вы можете скачать из OSM или найти в любом другом источнике границы административно территориальных границ России и пересечь с ними датасет с помощью `geopandas.sjoin` (этот вариант более надежный, но нужно будет разобраться с тем, как устроены границы АТД в OSM, обратите внимание на [этот тег](https://wiki.openstreetmap.org/wiki/Key:admin_level))


2. **Используйте данные OSM**: подумайте, какие объекты могут влиять на продажи фичерфонов. Гипотеза: такие телефоны покупают люди, приезжающие в город или страну ненадолго, чтобы вставить туда отдельную симкарту для роуминга. Можно попробовать использовать местоположения железнодорожных вокзалов (изучите [этот тег](https://wiki.openstreetmap.org/wiki/Tag:railway%3Dstation)). Необходимо использовать хотя бы 5 разных типов объектов из OSM. Скорее всего, вам придется качать данные OSM отдельно для разных городов (см. пример для Нью-Йорка из лекции)


3. **Используйте разные способы генерации признаков**: описать положение салона МегаФона относительно станций метро можно разными способами - найти ***расстояние до ближайшей станции***, или же посчитать, сколько станций попадает в ***500 метровую буферную зону*** вокруг салона. Такие признаки будут нести разную информацию. Так же попробуйте поэкспериментировать с размерами буферных зон (представьте, что значат в реальности радиусы 100, 500, 1000 метров). Попробуйте посчитать расстояние до центра города, до других объектов.

4. **Сделайте визуализации**: постройте 2-3 карты для какого нибудь из городов - как распределен в пространстве таргет, где находятся объекты, полученные вами из OSM. Можете использовать любой инструмент - обычный `plot()`, `folium`, `keplergl`. Если выберете Кеплер, обязательно сохраните в файл конфиг карты, чтобы ее можно было воспроизвести. Сделать это можно вот так:

```python
import json
json_data = kepler_map.config
with open('kepler_config.json', 'w') as outfile:
    json.dump(json_data, outfile)
```
5. Задание не ограничено приведенными выше пунктами, попробуйте нагенерировать интересных признаков, найти в интернете дополнительные данные (в таком случае в комментарии к коду укажите ссылку на ресурс, откуда взяли данные)



6. Это довольно сложная задача - датасет очень маленький, данные по своей природе довольно случайны. Поэтому место и скор на Kaggle не будут играть решающую роль в оценке, но позволят заработать дополнительные баллы

### Read data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
import pandas as pd
import numpy as np
import geojson
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [ ]:
#выгружаем данным (города)
#лучше всего - реализуй первым методом, это параша полная что я делала - в итоге тот же результат

#далее я тебе предлагю делать запросы по тегам в турбо, далее нажимаешь запустить, затем скачать json файл и прочитать, далее ориентироваться в геодатафрейме просто
#это пример запроса для городо - но лучше геокодером, еще раз))
"""
[out:json];area(3600060189)->.searchArea;
(node["place"~"city|town"](area.searchArea);); out body geom;
>;
out skel qt; 
"""

In [ ]:
#выгружаем данным (города)
#здесь я тебе предлагю сделать запрос в турбо, открыть json файл и прочитать просто

# в турбо забей такой запрос, запусти, скачай и прочитай
# но лучше всего - реализуй первым методом, это параша полная что я делала - в итоге тот же результат
"""
[out:json];area(3600060189)->.searchArea;
(node["place"~"city"](area.searchArea);); out body geom;
>;
out skel qt; 
"""
####################################################################3
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

nominatim = Nominatim()
areaId = nominatim.query('Russia').areaId()
overpass = Overpass()
query = overpassQueryBuilder(area=areaId, elementType=['rel'], selector='"place"~"city|town"', out='body geom;out skel qt',includeGeometry=True)
result = overpass.query(query,timeout=600)

[nominatim] downloading data: search
[overpass] downloading data: [timeout:600][out:json];area(3600060189)->.searchArea;(rel["place"~"city|town"](area.searchArea);); out body geom;out skel qt geom;


In [ ]:
##############################################################################3
#                                  это не надо тебе, если геокодером
##############################################################################
"""#"вытаскиваем" координаты из result
from shapely.geometry import Polygon
from shapely.geometry.multipolygon import MultiPolygon

result_dict = {}
mas = []
cnt = 0
for i in result.elements():
  data = {}
  try:
    data['name'] = i.tag('name')
    data['region'] = i.tag('addr:region')   
  except Exception as e:
    #print(e)
    pass  

  mass_polygon = []  
  try:
    for j in range(0,len(i.geometry()['coordinates'])):
      if len(i.geometry()['coordinates'][j])==1:
        for k in range(0,len(i.geometry()['coordinates'][j])):
          polygon = (Polygon(i.geometry()['coordinates'][j][k]))
      else:
          try:
            for k in range(0,len(i.geometry()['coordinates'][j])):
              polygon = (Polygon(i.geometry()['coordinates'][j][k]))
          except Exception as e:
            polygon = (Polygon(i.geometry()['coordinates'][j]))
      mass_polygon.append(polygon)
    data['geometry'] = MultiPolygon(mass_polygon)
    mas.append(data)
  except Exception as e:
    pass
    #print(e)
result_dict ["results"] = mas
gdf_polygons = gpd.GeoDataFrame(result_dict['results'])"""

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/')

In [ ]:
train = pd.read_csv('/content/gdrive/MyDrive/mf_geo_train.csv')
test = pd.read_csv('/content/gdrive/MyDrive/mf_geo_test.csv')

In [ ]:
#переводим в геопандас
gdf = gpd.GeoDataFrame(train, geometry=gpd.points_from_xy(train.lon, train.lat))
gdf_test = gpd.GeoDataFrame(test, geometry=gpd.points_from_xy(test.lon, test.lat))

In [ ]:
test.shape

In [ ]:
# тут я пересекаю, но за тебя метку города поставит геокодер, далее город - big_city
"""intersect = gpd.sjoin(gdf,gdf_polygons[['name','geometry']], op='within')
intersect = intersect.drop(['index_right'], axis = 1)
intersect.shape"""


In [ ]:
"""intersect_test = gpd.sjoin(gdf_test,gdf_polygons[['name','geometry']], op='within')
intersect_test = intersect_test.drop(['index_right'], axis = 1)
intersect_test.shape
"""

In [ ]:
"""dif = pd.concat([intersect[['point_id', 'lon'	,'lat','target','geometry']], gdf[['point_id', 'lon'	,'lat','target','geometry']]]).drop_duplicates(keep=False)
dif.plot(figsize=(15,10))"""

In [ ]:
"""dif_test = pd.concat([intersect_test[['point_id', 'lon'	,'lat','target','geometry']], gdf_test[['point_id', 'lon'	,'lat','target','geometry']]]).drop_duplicates(keep=False)"""

In [ ]:
"""#здесь для тех точек, которые "выпали" из нашей таблицы, записываем ближайший город
from shapely.geometry import Point, Polygon
import random
from operator import itemgetter

dif['name']=np.zeros(dif.shape[0])
dif = dif.reset_index()
cnt = 0
for j in dif['geometry']:
  min = gdf_polygons_short['geometry'][0][0]
  index_min = 0
  for i in range(len(gdf_polygons_short['geometry'])):
    for poly in gdf_polygons_short['geometry'][i]:
      if poly.distance(j) < min.distance(j):
        min=poly
        index_min = i
  dif['name'][cnt]= gdf_polygons_short['name'][index_min]
  cnt+=1"""

In [ ]:
"""dif_test['name']=np.zeros(dif_test.shape[0])
dif_test = dif_test.reset_index()
cnt = 0
for j in dif_test['geometry']:
  min = gdf_polygons_short['geometry'][0][0]
  index_min = 0
  for i in range(len(gdf_polygons_short['geometry'])):
    for poly in gdf_polygons_short['geometry'][i]:
      if poly.distance(j) < min.distance(j):
        min=poly
        index_min = i
  dif_test['name'][cnt]= gdf_polygons_short['name'][index_min]
  cnt+=1"""

In [ ]:
"""#теперь у нас все 425 точек
new_data = pd.concat([dif,intersect])
new_data = new_data.drop(['index'],axis = 1).reset_index()
new_data.shape"""

In [ ]:
"""#теперь у нас все 107 точек
new_data_test = pd.concat([dif_test,intersect_test])
new_data_test = new_data_test.drop(['index'],axis = 1).reset_index()
new_data_test.shape"""

In [ ]:
#определим расстояние до центра города
#предположу, что для точек, находящихся в таких городах, как Щербинка, будем считать до центра Москвы 
query = overpassQueryBuilder(area=areaId, elementType=['node'], selector='"place"~"city|town"', out='body geom;out skel qt',includeGeometry=True)
result_cities = overpass.query(query,timeout=600)

[overpass] downloading data: [timeout:600][out:json];area(3600060189)->.searchArea;(node["place"~"city|town"](area.searchArea);); out body geom;out skel qt geom;


In [ ]:
#дистанция до центра города
from geopy import distance
new_data['distance_centre']=np.zeros(new_data.shape[0])
for i in range(len(new_data['point_id'])):
  point_true = (new_data['geometry'][i].y,new_data['geometry'][i].x)
  for j in result_cities.elements():
    try:
      name = j.tag('name')
    except Exception as e:
      name = ""
    if new_data['big_city'][i] == name:
      x = j.lat()
      y = j.lon()
      point = (x,y)
      distance_new_data =  distance.distance(point_true,point).km
      index = new_data[new_data['point_id']==new_data['point_id'][i]].index.values.astype(int)[0]
      new_data['distance_centre'][index] = distance_new_data
 

In [ ]:
new_data = new_data.drop(['index'],axis = 1).reset_index()
new_data.head()

In [ ]:
new_data_test['distance_centre']=np.zeros(new_data_test.shape[0])
for i in range(len(new_data_test['point_id'])):
  point_true = (new_data_test['geometry'][i].y,new_data_test['geometry'][i].x)
  for j in result_cities.elements():
    try:
      name = j.tag('name')
    except Exception as e:
      name = ""
    if new_data_test['big_city'][i] == name:
      x = j.lat()
      y = j.lon()
      point = (x,y)
      distance_new_data =  distance.distance(point_true,point).km
      index = new_data_test[new_data_test['point_id']==new_data_test['point_id'][i]].index.values.astype(int)[0]
      new_data_test['distance_centre'][index] = distance_new_data

In [ ]:
new_data_test = new_data_test.drop(['index'],axis = 1).reset_index()
new_data_test.head()

In [ ]:
tuts_tuts = gpd.sjoin(new_data,adm[['name','geometry']], op='within')
tuts_tuts = tuts_tuts.drop(['index_right','index'],axis = 1)
tuts_tuts = tuts_tuts.rename(columns={"name_left": "city", "name_right": "region"}).reset_index()
tuts_tuts.head()

In [ ]:
tuts_tuts_test = gpd.sjoin(new_data_test,adm[['name','geometry']], op='within')
tuts_tuts_test = tuts_tuts_test.drop(['index_right','index'],axis = 1)
tuts_tuts_test = tuts_tuts_test.rename(columns={"name_left": "city", "name_right": "region"}).reset_index()
tuts_tuts_test.head()

In [ ]:
#рассчитаем расстояние до ближайшего ж/д вокзала

list_cities = ['Нижний Новгород', 'Москва', 'Ростов-на-Дону', 'Красноярск',
       'Санкт-Петербург', 'Уфа', 'Казань', 'Екатеринбург',
       'Новосибирск', 'Самара']

tuts_tuts['distance_railway']=np.zeros(tuts_tuts.shape[0])

def find_min(mass):
  min = mass[0]
  for i in mass:
    if i < min:
      min = i
  return min

for i in range(len(tuts_tuts['point_id'])):
      point_true = (tuts_tuts['geometry'][i].y,tuts_tuts['geometry'][i].x)
      mass_points =[]
      for ii in list_cities:        
        query = overpassQueryBuilder(area=nominatim.query(ii+',Россия').areaId(), elementType=['node'], selector='"railway"="station"', out='body geom;out skel qt',includeGeometry=True)
        result_cities = overpass.query(query,timeout=600)
        for j in result_cities.elements():
          try:
            if j.tag('station') == None:
              x = j.lat() 
              y = j.lon()
              point = (x,y)
              distance_new_data =  distance.distance(point_true,point).km
              #print(distance_new_data)
              mass_points.append(distance_new_data)
          except Exception as e:
              pass
      if len(mass_points)>0:
        min_dist = find_min(mass_points)
        index = tuts_tuts[tuts_tuts['point_id']==tuts_tuts['point_id'][i]].index.values.astype(int)[0]#БАТЯ
        tuts_tuts['distance_railway'][index] = min_dist

In [ ]:
list_cities = ['Нижний Новгород', 'Москва', 'Ростов-на-Дону', 'Красноярск',
       'Санкт-Петербург', 'Кронштадт', 'Уфа', 'Казань', 'Екатеринбург',
       'Новосибирск', 'Самара']

tuts_tuts_test['distance_railway']=np.zeros(tuts_tuts_test.shape[0])

def find_min(mass):
  min = mass[0]
  for i in mass:
    if i < min:
      min = i
  return min

for i in range(len(tuts_tuts_test['point_id'])):
      point_true = (tuts_tuts_test['geometry'][i].y,tuts_tuts_test['geometry'][i].x)
      mass_points =[]
      for ii in list_cities:        
        query = overpassQueryBuilder(area=nominatim.query(ii+',Россия').areaId(), elementType=['node'], selector='"railway"="station"', out='body geom;out skel qt',includeGeometry=True)
        result_cities = overpass.query(query,timeout=600)
        for j in result_cities.elements():
          try:
            if j.tag('station') == None:
              x = j.lat() 
              y = j.lon()
              point = (x,y)
              distance_new_data =  distance.distance(point_true,point).km
              #print(distance_new_data)
              mass_points.append(distance_new_data)
          except Exception as e:
              pass
      if len(mass_points)>0:
        min_dist = find_min(mass_points)
        index = tuts_tuts_test[tuts_tuts_test['point_id']==tuts_tuts_test['point_id'][i]].index.values.astype(int)[0]#БАТЯ
        tuts_tuts_test['distance_railway'][index] = min_dist

In [ ]:
#количество гостиниц в радиусе 0.5 км
tuts_tuts['counts_hotel_500']=np.zeros(tuts_tuts.shape[0])
radius = 0.5 # in kilometer
for i in range(len(tuts_tuts['point_id'])):
  query = overpassQueryBuilder(area=nominatim.query(tuts_tuts['big_city'][i]+',Россия').areaId(), elementType=['node'], selector='"tourism"~"hostel|motel|hostel"', out='body geom;out skel qt',includeGeometry=True)
  result_cities = overpass.query(query,timeout=600)
  center_point = [{'lat': tuts_tuts['geometry'][i].y, 'lng': tuts_tuts['geometry'][i].x}]
  center_point_tuple = tuple(center_point[0].values()) 
  cnt = 0
  for j in result_cities.elements():
    test_point = [{'lat': j.lat(), 'lng': j.lon()}]      
    test_point_tuple = tuple(test_point[0].values())
    dis = distance.distance(center_point_tuple, test_point_tuple).km
    if dis <= radius:
      cnt+=1
  tuts_tuts['counts_hotel_500'][i]=cnt
tuts_tuts.head()

In [ ]:
tuts_tuts_test['counts_hotel_500']=np.zeros(tuts_tuts_test.shape[0])
radius = 0.5 # in kilometer
for i in range(len(tuts_tuts_test['point_id'])):
  query = overpassQueryBuilder(area=nominatim.query(tuts_tuts_test['big_city'][i]+',Россия').areaId(), elementType=['node'], selector='"tourism"~"hostel|motel|hostel"', out='body geom;out skel qt',includeGeometry=True)
  result_cities = overpass.query(query,timeout=600)
  center_point = [{'lat': tuts_tuts_test['geometry'][i].y, 'lng': tuts_tuts_test['geometry'][i].x}]
  center_point_tuple = tuple(center_point[0].values())
  cnt = 0
  for j in result_cities.elements():
    test_point = [{'lat': j.lat(), 'lng': j.lon()}]      
    test_point_tuple = tuple(test_point[0].values())
    dis = distance.distance(center_point_tuple, test_point_tuple).km
    if dis <= radius:
      cnt+=1
  tuts_tuts_test['counts_hotel_500'][i]=cnt
tuts_tuts_test.head()

In [ ]:
#посчитаем количество остановок общественного трансорта в радиусе 500 м
#"public_transport"~"platform"
tuts_tuts['public_transport_500']=np.zeros(tuts_tuts.shape[0])
radius = 0.5 # in kilometer
for i in range(len(tuts_tuts['point_id'])):
  query = overpassQueryBuilder(area=nominatim.query(tuts_tuts['big_city'][i]+',Россия').areaId(), elementType='node', selector='"public_transport"~"platform"', out='body geom;out skel qt',includeGeometry=True)
  result_cities = overpass.query(query,timeout=600)
  center_point = [{'lat': tuts_tuts['geometry'][i].y, 'lng': tuts_tuts['geometry'][i].x}]
  center_point_tuple = tuple(center_point[0].values()) #
  cnt = 0
  #print(len(result_cities.elements()))
  for j in result_cities.elements():
    test_point = [{'lat': j.lat(), 'lng': j.lon()}]   
    test_point_tuple = tuple(test_point[0].values())
    #print(test_point_tuple) 
    dis = distance.distance(center_point_tuple, test_point_tuple).km
    if dis <= radius:
      cnt+=1
  tuts_tuts['public_transport_500'][i]=cnt
tuts_tuts.head()

In [ ]:
#"public_transport"~"platform"
tuts_tuts_test['public_transport_500']=np.zeros(tuts_tuts_test.shape[0])
radius = 0.5 # in kilometer
for i in range(len(tuts_tuts_test['point_id'])):
  query = overpassQueryBuilder(area=nominatim.query(tuts_tuts_test['big_city'][i]+',Россия').areaId(), elementType='node', selector='"public_transport"~"platform"', out='body geom;out skel qt',includeGeometry=True)
  result_cities = overpass.query(query,timeout=600)
  center_point = [{'lat': tuts_tuts_test['geometry'][i].y, 'lng': tuts_tuts_test['geometry'][i].x}]
  center_point_tuple = tuple(center_point[0].values()) #
  cnt = 0
  for j in result_cities.elements():
    test_point = [{'lat': j.lat(), 'lng': j.lon()}]   
    test_point_tuple = tuple(test_point[0].values())
    #print(test_point_tuple) 
    dis = distance.distance(center_point_tuple, test_point_tuple).km
    if dis <= radius:
      cnt+=1
  tuts_tuts_test['public_transport_500'][i]=cnt
tuts_tuts_test.head()

In [ ]:
#посчитаем количество мест общественного питания в радиусе 500 м
tuts_tuts['count_food']=np.zeros(tuts_tuts.shape[0])
radius = 0.5 # in kilometer
for i in range(len(tuts_tuts['point_id'])):
  query = overpassQueryBuilder(area=nominatim.query(tuts_tuts['big_city'][i]+',Россия').areaId(), elementType='node', selector='"amenity"~"fast_food|cafe"', out='body geom;out skel qt',includeGeometry=True)
  result_cities = overpass.query(query,timeout=600)
  center_point = [{'lat': tuts_tuts['geometry'][i].y, 'lng': tuts_tuts['geometry'][i].x}]
  center_point_tuple = tuple(center_point[0].values()) #
  cnt = 0
  for j in result_cities.elements():
    test_point = [{'lat': j.lat(), 'lng': j.lon()}]   
    test_point_tuple = tuple(test_point[0].values())
    dis = distance.distance(center_point_tuple, test_point_tuple).km
    if dis <= radius:
      cnt+=1
  tuts_tuts['count_food'][i]=cnt
tuts_tuts

In [ ]:
tuts_tuts_test['count_food']=np.zeros(tuts_tuts_test.shape[0])
radius = 0.5 # in kilometer
for i in range(len(tuts_tuts_test['point_id'])):
  query = overpassQueryBuilder(area=nominatim.query(tuts_tuts_test['big_city'][i]+',Россия').areaId(), elementType='node', selector='"amenity"~"fast_food|cafe"', out='body geom;out skel qt',includeGeometry=True)
  result_cities = overpass.query(query,timeout=600)
  center_point = [{'lat': tuts_tuts_test['geometry'][i].y, 'lng': tuts_tuts_test['geometry'][i].x}]
  center_point_tuple = tuple(center_point[0].values()) #
  cnt = 0
  for j in result_cities.elements():
    test_point = [{'lat': j.lat(), 'lng': j.lon()}]   
    test_point_tuple = tuple(test_point[0].values())
    dis = distance.distance(center_point_tuple, test_point_tuple).km
    if dis <= radius:
      cnt+=1
  tuts_tuts_test['count_food'][i]=cnt
tuts_tuts_test

In [ ]:
tuts_tuts = tuts_tuts.drop(['city','lon', 'lat', 'geometry'], axis = 1)
tuts_tuts_test = tuts_tuts_test.drop(['city','lon', 'lat', 'geometry'], axis = 1)
tuts_tuts.head()

### Fit model

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(tuts_tuts.drop('target', axis=1), tuts_tuts[['target']])
model = LinearRegression().fit(X_train.drop('point_id', axis=1), y_train)
y_predict =  model.predict(X_valid.drop('point_id', axis=1))
mean_absolute_error(y_valid, model.predict(X_valid.drop('point_id', axis=1)))

In [ ]:
targ = pd.DataFrame(tuts_tuts_test['point_id'])
targ['target'] = model.predict(tuts_tuts_test.drop(['point_id', 'target'], axis=1))

targ.to_csv('g1_vyb.csv', encoding = 'utf-8', index=False)
targ

### Make submission

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission['target'] = model.predict(X_valid.drop('point_id', axis=1))
submission.to_csv('my_submission_01.csv', index=False)